<a href="https://colab.research.google.com/github/anonymouaccount/HTGNNs/blob/main/HTGNN_Experimental_Results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install torch torch-geometric numpy matplotlib scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 71.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 76.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 60.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12


In [1]:
!pip install torch-geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.9 MB/s eta 0:00:00


# **HTGNN_houses**

In [3]:
# Mounting Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Import necessary libraries
import os
import pandas as pd
import numpy as np
import networkx as nx
import torch
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data, DataLoader
import torch.optim as optim

# Set environment variables
os.environ['LC_ALL'] = 'en_US.UTF-8'

# Load the dataset
ratings = pd.read_csv('/content/drive/MyDrive/user_activity.csv')

# Convert timestamp to datetime and sort by timestamp
ratings['timestamp'] = pd.to_datetime(ratings['create_timestamp'])
ratings = ratings.sort_values(by='timestamp')

# Split the data into train and test sets
train_data, test_data = train_test_split(ratings, test_size=0.2, shuffle=False)

# Function to create graph from data
def create_graph(data):
    G = nx.DiGraph()
    for _, row in data.iterrows():
        # Use the timestamp() method to get a numeric timestamp
        G.add_edge(row['user_id'], row['item_id'], timestamp=row['timestamp'].timestamp())
    return G

train_graph = create_graph(train_data)
test_graph = create_graph(test_data)

# Function to convert NetworkX graph to PyTorch Geometric Data object
def convert_to_pyg_data(graph, num_features=8):
    nodes = list(graph.nodes())
    node_mapping = {node: i for i, node in enumerate(nodes)}
    edge_index = torch.tensor([[node_mapping[u], node_mapping[v]] for u, v in graph.edges]).t().contiguous()
    edge_time = torch.tensor([graph[u][v]['timestamp'] for u, v in graph.edges], dtype=torch.float)

    # Create a feature matrix with fixed number of features per node
    x = torch.randn(len(nodes), num_features)

    # Random labels for the nodes (binary classification: 0 or 1)
    y = torch.randint(0, 2, (len(nodes),))

    data = Data(x=x, edge_index=edge_index, edge_time=edge_time, y=y)
    return data

train_data_pyg = convert_to_pyg_data(train_graph)
test_data_pyg = convert_to_pyg_data(test_graph)

train_loader = DataLoader([train_data_pyg], batch_size=1, shuffle=True)
test_loader = DataLoader([test_data_pyg], batch_size=1, shuffle=False)

# HTGNN Model definition
class HTGNN(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(HTGNN, self).__init__()
        self.conv1 = GCNConv(in_channels, 8)
        self.conv2 = GCNConv(8 + 8, out_channels)
        self.time_embedding = torch.nn.Embedding(365, 8)  # Embedding for time

    def forward(self, x, edge_index, edge_time):
        x = self.conv1(x, edge_index)
        x = F.relu(x)

        # Embedding for edge times
        time_embeds = self.time_embedding((edge_time.long() % 365).view(-1, 1)).view(-1, 8)

        # Average the edge time embeddings per node
        node_time_embeds = torch.zeros_like(x)
        for i in range(edge_index.size(1)):
            node_time_embeds[edge_index[0, i]] += time_embeds[i]

        x = torch.cat([x, node_time_embeds], dim=1)
        x = self.conv2(x, edge_index)
        return x

# Initialize the model, loss function, and optimizer
model = HTGNN(in_channels=train_data_pyg.num_node_features, out_channels=2)
optimizer = optim.Adam(model.parameters(), lr=0.01)
loss_fn = torch.nn.CrossEntropyLoss()

# Training function
def train(model, loader, optimizer, loss_fn):
    model.train()
    total_loss = 0
    for data in loader:
        optimizer.zero_grad()
        out = model(data.x, data.edge_index, data.edge_time)
        loss = loss_fn(out, data.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

# Evaluation function
def evaluate(model, loader):
    model.eval()
    correct = 0
    for data in loader:
        out = model(data.x, data.edge_index, data.edge_time)
        pred = out.argmax(dim=1)
        correct += (pred == data.y).sum().item()
    return correct / len(loader.dataset)

# Training loop (now running for only 10 epochs)
for epoch in range(10):  # Update: Loop only for 10 epochs
    train_loss = train(model, train_loader, optimizer, loss_fn)
    test_acc = evaluate(model, test_loader)
    print(f'Epoch {epoch+1}, Loss: {train_loss:.4f}, Test Accuracy: {test_acc:.4f}')

# Additional evaluation metrics
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# Function to calculate MRR
def mrr_score(y_true, y_pred):
    order = np.argsort(y_pred)[::-1]
    ranks = np.where(y_true[order] == 1)[0] + 1
    return np.mean(1.0 / ranks)

# Function to calculate NDCG
def ndcg_score(y_true, y_pred, k=10):
    order = np.argsort(y_pred)[::-1]
    y_true = np.take(y_true, order[:k])

    gains = 2 ** y_true - 1
    discounts = np.log2(np.arange(2, k + 2))
    dcg = np.sum(gains / discounts)

    ideal_gains = 2 ** np.sort(y_true)[::-1] - 1
    idcg = np.sum(ideal_gains / discounts)

    return dcg / idcg if idcg > 0 else 0.0

# Evaluation function with metrics
def evaluate_with_metrics(model, loader):
    model.eval()
    all_preds = []
    all_labels = []
    for data in loader:
        out = model(data.x, data.edge_index, data.edge_time)
        pred = out.argmax(dim=1)
        all_preds.append(pred.detach().cpu().numpy())
        all_labels.append(data.y.detach().cpu().numpy())

    all_preds = np.concatenate(all_preds)
    all_labels = np.concatenate(all_labels)

    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='macro')
    recall = recall_score(all_labels, all_preds, average='macro')
    f1 = f1_score(all_labels, all_preds, average='macro')
    mrr = mrr_score(all_labels, all_preds)
    ndcg = ndcg_score(all_labels, all_preds)

    return accuracy, precision, recall, f1, mrr, ndcg

# Final evaluation
accuracy, precision, recall, f1, mrr, ndcg = evaluate_with_metrics(model, test_loader)
print(f'NDCG: {ndcg:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1-Score: {f1:.4f}')


Mounted at /content/drive


/usr/local/lib/python3.11/dist-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Epoch 1, Loss: 8.9408, Test Accuracy: 2311.0000
Epoch 2, Loss: 7.3028, Test Accuracy: 2305.0000
Epoch 3, Loss: 6.2018, Test Accuracy: 2294.0000
Epoch 4, Loss: 5.8263, Test Accuracy: 2252.0000
Epoch 5, Loss: 5.8350, Test Accuracy: 2250.0000
Epoch 6, Loss: 5.7554, Test Accuracy: 2253.0000
Epoch 7, Loss: 5.5001, Test Accuracy: 2253.0000
Epoch 8, Loss: 5.1183, Test Accuracy: 2241.0000
Epoch 9, Loss: 4.6827, Test Accuracy: 2246.0000
Epoch 10, Loss: 4.2610, Test Accuracy: 2254.0000
NDCG: 0.7179, Precision: 0.4863, Recall: 0.4865, F1-Score: 0.4845


# HTGNN-**movies**

In [45]:
# Mounting Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Import necessary libraries
import os
import pandas as pd
import numpy as np
import networkx as nx
import torch
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data, DataLoader
import torch.optim as optim

# Set environment variables
os.environ['LC_ALL'] = 'en_US.UTF-8'

# Load the dataset
ratings = pd.read_csv('/content/drive/MyDrive/movielens/ratings_small.csv')

# Convert timestamp to datetime and sort by timestamp
ratings['timestamp'] = pd.to_datetime(ratings['timestamp'], unit='s')
ratings = ratings.sort_values(by='timestamp')

# Split the data into train and test sets
train_data, test_data = train_test_split(ratings, test_size=0.2, shuffle=False)

# Function to create graph from data
def create_graph(data):
    G = nx.DiGraph()
    for _, row in data.iterrows():
        G.add_edge(row['userId'], row['movieId'], timestamp=row['timestamp'].timestamp())
    return G

train_graph = create_graph(train_data)
test_graph = create_graph(test_data)

# Function to convert NetworkX graph to PyTorch Geometric Data object
def convert_to_pyg_data(graph, num_features=8):
    nodes = list(graph.nodes())
    node_mapping = {node: i for i, node in enumerate(nodes)}
    edge_index = torch.tensor([[node_mapping[u], node_mapping[v]] for u, v in graph.edges]).t().contiguous()
    edge_time = torch.tensor([graph[u][v]['timestamp'] for u, v in graph.edges], dtype=torch.float)

    # Create a feature matrix with fixed number of features per node
    x = torch.randn(len(nodes), num_features)

    # Random labels for the nodes (binary classification: 0 or 1)
    y = torch.randint(0, 2, (len(nodes),))

    data = Data(x=x, edge_index=edge_index, edge_time=edge_time, y=y)
    return data

train_data_pyg = convert_to_pyg_data(train_graph)
test_data_pyg = convert_to_pyg_data(test_graph)

train_loader = DataLoader([train_data_pyg], batch_size=1, shuffle=True)
test_loader = DataLoader([test_data_pyg], batch_size=1, shuffle=False)

# HTGNN Model definition
class HTGNN(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(HTGNN, self).__init__()
        self.conv1 = GCNConv(in_channels, 8)
        self.conv2 = GCNConv(8 + 8, out_channels)
        self.time_embedding = torch.nn.Embedding(365, 8)  # Embedding for time

    def forward(self, x, edge_index, edge_time):
        x = self.conv1(x, edge_index)
        x = F.relu(x)

        # Embedding for edge times
        time_embeds = self.time_embedding((edge_time.long() % 365).view(-1, 1)).view(-1, 8)

        # Average the edge time embeddings per node
        node_time_embeds = torch.zeros_like(x)
        for i in range(edge_index.size(1)):
            node_time_embeds[edge_index[0, i]] += time_embeds[i]

        x = torch.cat([x, node_time_embeds], dim=1)
        x = self.conv2(x, edge_index)
        return x

# Initialize the model, loss function, and optimizer
model = HTGNN(in_channels=train_data_pyg.num_node_features, out_channels=2)
optimizer = optim.Adam(model.parameters(), lr=0.01)
loss_fn = torch.nn.CrossEntropyLoss()

# Training function
def train(model, loader, optimizer, loss_fn):
    model.train()
    total_loss = 0
    for data in loader:
        optimizer.zero_grad()
        out = model(data.x, data.edge_index, data.edge_time)
        loss = loss_fn(out, data.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

# Evaluation function
def evaluate(model, loader):
    model.eval()
    correct = 0
    for data in loader:
        out = model(data.x, data.edge_index, data.edge_time)
        pred = out.argmax(dim=1)
        correct += (pred == data.y).sum().item()
    return correct / len(loader.dataset)

# Training loop (now running for only 10 epochs)
for epoch in range(10):  # Update: Loop only for 10 epochs
    train_loss = train(model, train_loader, optimizer, loss_fn)
    test_acc = evaluate(model, test_loader)
    print(f'Epoch {epoch+1}, Loss: {train_loss:.4f}, Test Accuracy: {test_acc:.4f}')

# Additional evaluation metrics
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# Function to calculate MRR
def mrr_score(y_true, y_pred):
    order = np.argsort(y_pred)[::-1]
    ranks = np.where(y_true[order] == 1)[0] + 1
    return np.mean(1.0 / ranks)

# Function to calculate NDCG
def ndcg_score(y_true, y_pred, k=10):
    order = np.argsort(y_pred)[::-1]
    y_true = np.take(y_true, order[:k])

    gains = 2 ** y_true - 1
    discounts = np.log2(np.arange(2, k + 2))
    dcg = np.sum(gains / discounts)

    ideal_gains = 2 ** np.sort(y_true)[::-1] - 1
    idcg = np.sum(ideal_gains / discounts)

    return dcg / idcg if idcg > 0 else 0.0

# Evaluation function with metrics
def evaluate_with_metrics(model, loader):
    model.eval()
    all_preds = []
    all_labels = []
    for data in loader:
        out = model(data.x, data.edge_index, data.edge_time)
        pred = out.argmax(dim=1)
        all_preds.append(pred.detach().cpu().numpy())
        all_labels.append(data.y.detach().cpu().numpy())

    all_preds = np.concatenate(all_preds)
    all_labels = np.concatenate(all_labels)

    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='macro')
    recall = recall_score(all_labels, all_preds, average='macro')
    f1 = f1_score(all_labels, all_preds, average='macro')
    mrr = mrr_score(all_labels, all_preds)
    ndcg = ndcg_score(all_labels, all_preds)

    return accuracy, precision, recall, f1, mrr, ndcg

# Final evaluation
accuracy, precision, recall, f1, mrr, ndcg = evaluate_with_metrics(model, test_loader)
print(f'NDCG: {ndcg:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1-Score: {f1:.4f}')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.11/dist-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Epoch 1, Loss: 15.0247, Test Accuracy: 2429.0000
Epoch 2, Loss: 12.1960, Test Accuracy: 2442.0000
Epoch 3, Loss: 12.8334, Test Accuracy: 2434.0000
Epoch 4, Loss: 11.3649, Test Accuracy: 2420.0000
Epoch 5, Loss: 9.9414, Test Accuracy: 2413.0000
Epoch 6, Loss: 9.9673, Test Accuracy: 2419.0000
Epoch 7, Loss: 9.7144, Test Accuracy: 2414.0000
Epoch 8, Loss: 8.7053, Test Accuracy: 2422.0000
Epoch 9, Loss: 7.8673, Test Accuracy: 2428.0000
Epoch 10, Loss: 7.6751, Test Accuracy: 2429.0000
NDCG: 0.9748, Precision: 0.5007, Recall: 0.5007, F1-Score: 0.4987


# HTGNN-**amazon**

In [1]:
# Mounting Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Import necessary libraries
import os
import pandas as pd
import numpy as np
import networkx as nx
import torch
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data, DataLoader
import torch.optim as optim

# Set environment variables
os.environ['LC_ALL'] = 'en_US.UTF-8'

# Load the dataset
ratings = pd.read_csv('/content/drive/MyDrive/ratings_Beauty.csv')

# Display the first few rows to check the structure of the dataset
print(ratings.head())

ratings['timestamp'] = pd.to_datetime(ratings['Timestamp'], unit='s')
ratings = ratings.sort_values(by='timestamp')

# Split the data into train and test sets
train_data, test_data = train_test_split(ratings, test_size=0.2, shuffle=False)

train_data = train_data.sample(frac=0.1, random_state=42)
test_data = test_data.sample(frac=0.1, random_state=42)

# Function to create graph from data
def create_graph(data):
    G = nx.DiGraph()
    for _, row in data.iterrows():
        G.add_edge(row['UserId'], row['ProductId'], timestamp=row['Timestamp'])
    return G

train_graph = create_graph(train_data)
test_graph = create_graph(test_data)

# Function to convert NetworkX graph to PyTorch Geometric Data object
def convert_to_pyg_data(graph, num_features=8):
    nodes = list(graph.nodes())
    node_mapping = {node: i for i, node in enumerate(nodes)}
    edge_index = torch.tensor([[node_mapping[u], node_mapping[v]] for u, v in graph.edges]).t().contiguous()
    edge_time = torch.tensor([graph[u][v]['timestamp'] for u, v in graph.edges], dtype=torch.float)

    # Create a feature matrix with reduced number of features per node (e.g., 4 features instead of 8)
    x = torch.randn(len(nodes), num_features)

    # Random labels for the nodes (binary classification: 0 or 1)
    y = torch.randint(0, 2, (len(nodes),))

    data = Data(x=x, edge_index=edge_index, edge_time=edge_time, y=y)
    return data

# Convert graph data to PyTorch Geometric Data format
train_data_pyg = convert_to_pyg_data(train_graph, num_features=4)  # Reduced features
test_data_pyg = convert_to_pyg_data(test_graph, num_features=4)  # Reduced features

# Create data loaders with smaller batch size
train_loader = DataLoader([train_data_pyg], batch_size=1, shuffle=True)  # Batch size 1
test_loader = DataLoader([test_data_pyg], batch_size=1, shuffle=False)  # Batch size 1

# HTGNN Model definition
class HTGNN(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(HTGNN, self).__init__()
        self.conv1 = GCNConv(in_channels, 8)
        self.conv2 = GCNConv(8 + 8, out_channels)
        self.time_embedding = torch.nn.Embedding(365, 8)  # Embedding for time

    def forward(self, x, edge_index, edge_time):
        x = self.conv1(x, edge_index)
        x = F.relu(x)

        # Embedding for edge times
        time_embeds = self.time_embedding((edge_time.long() % 365).view(-1, 1)).view(-1, 8)

        # Average the edge time embeddings per node
        node_time_embeds = torch.zeros_like(x)
        for i in range(edge_index.size(1)):
            node_time_embeds[edge_index[0, i]] += time_embeds[i]

        x = torch.cat([x, node_time_embeds], dim=1)
        x = self.conv2(x, edge_index)
        return x

# Initialize the model, loss function, and optimizer
model = HTGNN(in_channels=train_data_pyg.num_node_features, out_channels=2)
optimizer = optim.Adam(model.parameters(), lr=0.01)
loss_fn = torch.nn.CrossEntropyLoss()

# Training function
def train(model, loader, optimizer, loss_fn):
    model.train()
    total_loss = 0
    for data in loader:
        optimizer.zero_grad()
        out = model(data.x, data.edge_index, data.edge_time)
        loss = loss_fn(out, data.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

# Evaluation function
def evaluate(model, loader):
    model.eval()
    correct = 0
    for data in loader:
        out = model(data.x, data.edge_index, data.edge_time)
        pred = out.argmax(dim=1)
        correct += (pred == data.y).sum().item()
    return correct / len(loader.dataset)

# Training loop (now running for only 10 epochs)
for epoch in range(10):  # Update: Loop only for 10 epochs
    train_loss = train(model, train_loader, optimizer, loss_fn)
    test_acc = evaluate(model, test_loader)
    print(f'Epoch {epoch+1}, Loss: {train_loss:.4f}, Test Accuracy: {test_acc:.4f}')

# Additional evaluation metrics
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# Function to calculate MRR
def mrr_score(y_true, y_pred):
    order = np.argsort(y_pred)[::-1]
    ranks = np.where(y_true[order] == 1)[0] + 1
    return np.mean(1.0 / ranks)

# Function to calculate NDCG
def ndcg_score(y_true, y_pred, k=10):
    order = np.argsort(y_pred)[::-1]
    y_true = np.take(y_true, order[:k])

    gains = 2 ** y_true - 1
    discounts = np.log2(np.arange(2, k + 2))
    dcg = np.sum(gains / discounts)

    ideal_gains = 2 ** np.sort(y_true)[::-1] - 1
    idcg = np.sum(ideal_gains / discounts)

    return dcg / idcg if idcg > 0 else 0.0

# Evaluation function with metrics
def evaluate_with_metrics(model, loader):
    model.eval()
    all_preds = []
    all_labels = []
    for data in loader:
        out = model(data.x, data.edge_index, data.edge_time)
        pred = out.argmax(dim=1)
        all_preds.append(pred.detach().cpu().numpy())
        all_labels.append(data.y.detach().cpu().numpy())

    all_preds = np.concatenate(all_preds)
    all_labels = np.concatenate(all_labels)

    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='macro')
    recall = recall_score(all_labels, all_preds, average='macro')
    f1 = f1_score(all_labels, all_preds, average='macro')
    mrr = mrr_score(all_labels, all_preds)
    ndcg = ndcg_score(all_labels, all_preds)

    return accuracy, precision, recall, f1, mrr, ndcg

# Final evaluation
accuracy, precision, recall, f1, mrr, ndcg = evaluate_with_metrics(model, test_loader)
print(f'NDCG: {ndcg:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1-Score: {f1:.4f}')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
           UserId   ProductId  Rating   Timestamp
0  A39HTATAQ9V7YF  0205616461     5.0  1369699200
1  A3JM6GV9MNOF9X  0558925278     3.0  1355443200
2  A1Z513UWSAAO0F  0558925278     5.0  1404691200
3  A1WMRR494NWEWV  0733001998     4.0  1382572800
4  A3IAAVS479H7M7  0737104473     1.0  1274227200


/usr/local/lib/python3.11/dist-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Epoch 1, Loss: 0.9077, Test Accuracy: 30768.0000
Epoch 2, Loss: 0.8840, Test Accuracy: 30821.0000
Epoch 3, Loss: 0.8630, Test Accuracy: 30778.0000
Epoch 4, Loss: 0.8443, Test Accuracy: 30818.0000
Epoch 5, Loss: 0.8276, Test Accuracy: 30829.0000
Epoch 6, Loss: 0.8127, Test Accuracy: 30815.0000
Epoch 7, Loss: 0.7994, Test Accuracy: 30838.0000
Epoch 8, Loss: 0.7877, Test Accuracy: 30821.0000
Epoch 9, Loss: 0.7773, Test Accuracy: 30802.0000
Epoch 10, Loss: 0.7681, Test Accuracy: 30801.0000
NDCG: 0.8878, Precision: 0.5014, Recall: 0.5014, F1-Score: 0.5014


In [54]:
print(ratings.columns)


Index(['UserId', 'ProductId', 'Rating', 'Timestamp'], dtype='object')


# GraphSAGE-**houses**

In [43]:
# Mounting Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Import necessary libraries
import os
import pandas as pd
import numpy as np
import networkx as nx
import torch
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from torch_geometric.nn import SAGEConv  # Use SAGEConv for GraphSAGE
from torch_geometric.data import Data, DataLoader
import torch.optim as optim

# Set environment variables
os.environ['LC_ALL'] = 'en_US.UTF-8'

# Load the dataset
ratings = pd.read_csv('/content/drive/MyDrive/user_activity.csv')

# Convert timestamp to datetime and sort by timestamp
ratings['timestamp'] = pd.to_datetime(ratings['create_timestamp'])
ratings = ratings.sort_values(by='timestamp')

# Split the data into train and test sets
train_data, test_data = train_test_split(ratings, test_size=0.2, shuffle=False)

# Function to create graph from data
def create_graph(data):
    G = nx.DiGraph()
    for _, row in data.iterrows():
        # Use the timestamp() method to get a numeric timestamp
        G.add_edge(row['user_id'], row['item_id'], timestamp=row['timestamp'].timestamp())
    return G

train_graph = create_graph(train_data)
test_graph = create_graph(test_data)

# Function to convert NetworkX graph to PyTorch Geometric Data object
def convert_to_pyg_data(graph, num_features=8):
    nodes = list(graph.nodes())
    node_mapping = {node: i for i, node in enumerate(nodes)}
    edge_index = torch.tensor([[node_mapping[u], node_mapping[v]] for u, v in graph.edges]).t().contiguous()
    edge_time = torch.tensor([graph[u][v]['timestamp'] for u, v in graph.edges], dtype=torch.float)

    # Create a feature matrix with fixed number of features per node
    x = torch.randn(len(nodes), num_features)

    # Random labels for the nodes (binary classification: 0 or 1)
    y = torch.randint(0, 2, (len(nodes),))

    data = Data(x=x, edge_index=edge_index, edge_time=edge_time, y=y)
    return data

train_data_pyg = convert_to_pyg_data(train_graph)
test_data_pyg = convert_to_pyg_data(test_graph)

train_loader = DataLoader([train_data_pyg], batch_size=1, shuffle=True)
test_loader = DataLoader([test_data_pyg], batch_size=1, shuffle=False)

# GraphSAGE Model definition
class GraphSAGE(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GraphSAGE, self).__init__()
        self.conv1 = SAGEConv(in_channels, 8)  # First GraphSAGE layer
        self.conv2 = SAGEConv(8, out_channels)  # Second GraphSAGE layer

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return x

# Initialize the model, loss function, and optimizer
model = GraphSAGE(in_channels=train_data_pyg.num_node_features, out_channels=2)
optimizer = optim.Adam(model.parameters(), lr=0.01)
loss_fn = torch.nn.CrossEntropyLoss()

# Training function
def train(model, loader, optimizer, loss_fn):
    model.train()
    total_loss = 0
    for data in loader:
        optimizer.zero_grad()
        out = model(data.x, data.edge_index)  # GraphSAGE does not use edge_time
        loss = loss_fn(out, data.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

# Evaluation function
def evaluate(model, loader):
    model.eval()
    correct = 0
    for data in loader:
        out = model(data.x, data.edge_index)  # GraphSAGE does not use edge_time
        pred = out.argmax(dim=1)
        correct += (pred == data.y).sum().item()
    return correct / len(loader.dataset)

# Training loop (now running for only 10 epochs)
for epoch in range(10):  # Update: Loop only for 10 epochs
    train_loss = train(model, train_loader, optimizer, loss_fn)
    test_acc = evaluate(model, test_loader)
    print(f'Epoch {epoch+1}, Loss: {train_loss:.4f}, Test Accuracy: {test_acc:.4f}')

# Additional evaluation metrics
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# Function to calculate MRR
def mrr_score(y_true, y_pred):
    order = np.argsort(y_pred)[::-1]
    ranks = np.where(y_true[order] == 1)[0] + 1
    return np.mean(1.0 / ranks)

# Function to calculate NDCG
def ndcg_score(y_true, y_pred, k=10):
    order = np.argsort(y_pred)[::-1]
    y_true = np.take(y_true, order[:k])

    gains = 2 ** y_true - 1
    discounts = np.log2(np.arange(2, k + 2))
    dcg = np.sum(gains / discounts)

    ideal_gains = 2 ** np.sort(y_true)[::-1] - 1
    idcg = np.sum(ideal_gains / discounts)

    return dcg / idcg if idcg > 0 else 0.0

# Evaluation function with metrics
def evaluate_with_metrics(model, loader):
    model.eval()
    all_preds = []
    all_labels = []
    for data in loader:
        out = model(data.x, data.edge_index)  # GraphSAGE does not use edge_time
        pred = out.argmax(dim=1)
        all_preds.append(pred.detach().cpu().numpy())
        all_labels.append(data.y.detach().cpu().numpy())

    all_preds = np.concatenate(all_preds)
    all_labels = np.concatenate(all_labels)

    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='macro')
    recall = recall_score(all_labels, all_preds, average='macro')
    f1 = f1_score(all_labels, all_preds, average='macro')
    mrr = mrr_score(all_labels, all_preds)
    ndcg = ndcg_score(all_labels, all_preds)

    return accuracy, precision, recall, f1, mrr, ndcg

# Final evaluation
accuracy, precision, recall, f1, mrr, ndcg = evaluate_with_metrics(model, test_loader)
print(f'NDCG: {ndcg:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1-Score: {f1:.4f}')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.11/dist-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Epoch 1, Loss: 0.7063, Test Accuracy: 2335.0000
Epoch 2, Loss: 0.7010, Test Accuracy: 2308.0000
Epoch 3, Loss: 0.6979, Test Accuracy: 2340.0000
Epoch 4, Loss: 0.6964, Test Accuracy: 2318.0000
Epoch 5, Loss: 0.6958, Test Accuracy: 2313.0000
Epoch 6, Loss: 0.6954, Test Accuracy: 2310.0000
Epoch 7, Loss: 0.6951, Test Accuracy: 2309.0000
Epoch 8, Loss: 0.6947, Test Accuracy: 2348.0000
Epoch 9, Loss: 0.6943, Test Accuracy: 2376.0000
Epoch 10, Loss: 0.6940, Test Accuracy: 2369.0000
NDCG: 0.5641, Precision: 0.5133, Recall: 0.5132, F1-Score: 0.5118


# GraphSAGE-**movies**

In [48]:
# Mounting Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Import necessary libraries
import os
import pandas as pd
import numpy as np
import networkx as nx
import torch
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from torch_geometric.nn import SAGEConv  # Use SAGEConv for GraphSAGE
from torch_geometric.data import Data, DataLoader
import torch.optim as optim

# Set environment variables
os.environ['LC_ALL'] = 'en_US.UTF-8'

# Load the MovieLens dataset
ratings = pd.read_csv('/content/drive/MyDrive/movielens/ratings_small.csv')

# Convert timestamp to datetime and sort by timestamp
ratings['timestamp'] = pd.to_datetime(ratings['timestamp'])
ratings = ratings.sort_values(by='timestamp')

# Split the data into train and test sets
train_data, test_data = train_test_split(ratings, test_size=0.2, shuffle=False)

# Function to create graph from data
def create_graph(data):
    G = nx.DiGraph()
    for _, row in data.iterrows():
        # Use the timestamp() method to get a numeric timestamp
        G.add_edge(row['userId'], row['movieId'], timestamp=row['timestamp'].timestamp())
    return G

train_graph = create_graph(train_data)
test_graph = create_graph(test_data)

# Function to convert NetworkX graph to PyTorch Geometric Data object
def convert_to_pyg_data(graph, num_features=8):
    nodes = list(graph.nodes())
    node_mapping = {node: i for i, node in enumerate(nodes)}
    edge_index = torch.tensor([[node_mapping[u], node_mapping[v]] for u, v in graph.edges]).t().contiguous()
    edge_time = torch.tensor([graph[u][v]['timestamp'] for u, v in graph.edges], dtype=torch.float)

    # Create a feature matrix with fixed number of features per node
    x = torch.randn(len(nodes), num_features)

    # Random labels for the nodes (binary classification: 0 or 1)
    y = torch.randint(0, 2, (len(nodes),))

    data = Data(x=x, edge_index=edge_index, edge_time=edge_time, y=y)
    return data

train_data_pyg = convert_to_pyg_data(train_graph)
test_data_pyg = convert_to_pyg_data(test_graph)

train_loader = DataLoader([train_data_pyg], batch_size=1, shuffle=True)
test_loader = DataLoader([test_data_pyg], batch_size=1, shuffle=False)

# GraphSAGE Model definition
class GraphSAGE(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GraphSAGE, self).__init__()
        self.conv1 = SAGEConv(in_channels, 8)  # First GraphSAGE layer
        self.conv2 = SAGEConv(8, out_channels)  # Second GraphSAGE layer

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return x

# Initialize the model, loss function, and optimizer
model = GraphSAGE(in_channels=train_data_pyg.num_node_features, out_channels=2)
optimizer = optim.Adam(model.parameters(), lr=0.01)
loss_fn = torch.nn.CrossEntropyLoss()

# Training function
def train(model, loader, optimizer, loss_fn):
    model.train()
    total_loss = 0
    for data in loader:
        optimizer.zero_grad()
        out = model(data.x, data.edge_index)  # GraphSAGE does not use edge_time
        loss = loss_fn(out, data.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

# Evaluation function
def evaluate(model, loader):
    model.eval()
    correct = 0
    for data in loader:
        out = model(data.x, data.edge_index)  # GraphSAGE does not use edge_time
        pred = out.argmax(dim=1)
        correct += (pred == data.y).sum().item()
    return correct / len(loader.dataset)

# Training loop (now running for only 10 epochs)
for epoch in range(10):  # Update: Loop only for 10 epochs
    train_loss = train(model, train_loader, optimizer, loss_fn)
    test_acc = evaluate(model, test_loader)
    print(f'Epoch {epoch+1}, Loss: {train_loss:.4f}, Test Accuracy: {test_acc:.4f}')

# Additional evaluation metrics
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# Function to calculate MRR
def mrr_score(y_true, y_pred):
    order = np.argsort(y_pred)[::-1]
    ranks = np.where(y_true[order] == 1)[0] + 1
    return np.mean(1.0 / ranks)

# Function to calculate NDCG
def ndcg_score(y_true, y_pred, k=10):
    order = np.argsort(y_pred)[::-1]
    y_true = np.take(y_true, order[:k])

    gains = 2 ** y_true - 1
    discounts = np.log2(np.arange(2, k + 2))
    dcg = np.sum(gains / discounts)

    ideal_gains = 2 ** np.sort(y_true)[::-1] - 1
    idcg = np.sum(ideal_gains / discounts)

    return dcg / idcg if idcg > 0 else 0.0

# Evaluation function with metrics
def evaluate_with_metrics(model, loader):
    model.eval()
    all_preds = []
    all_labels = []
    for data in loader:
        out = model(data.x, data.edge_index)  # GraphSAGE does not use edge_time
        pred = out.argmax(dim=1)
        all_preds.append(pred.detach().cpu().numpy())
        all_labels.append(data.y.detach().cpu().numpy())

    all_preds = np.concatenate(all_preds)
    all_labels = np.concatenate(all_labels)

    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='macro')
    recall = recall_score(all_labels, all_preds, average='macro')
    f1 = f1_score(all_labels, all_preds, average='macro')
    mrr = mrr_score(all_labels, all_preds)
    ndcg = ndcg_score(all_labels, all_preds)

    return accuracy, precision, recall, f1, mrr, ndcg

# Final evaluation
accuracy, precision, recall, f1, mrr, ndcg = evaluate_with_metrics(model, test_loader)
print(f'NDCG: {ndcg:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1-Score: {f1:.4f}')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.11/dist-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Epoch 1, Loss: 0.7851, Test Accuracy: 2420.0000
Epoch 2, Loss: 0.7560, Test Accuracy: 2435.0000
Epoch 3, Loss: 0.7338, Test Accuracy: 2439.0000
Epoch 4, Loss: 0.7183, Test Accuracy: 2421.0000
Epoch 5, Loss: 0.7086, Test Accuracy: 2405.0000
Epoch 6, Loss: 0.7038, Test Accuracy: 2420.0000
Epoch 7, Loss: 0.7025, Test Accuracy: 2426.0000
Epoch 8, Loss: 0.7031, Test Accuracy: 2414.0000
Epoch 9, Loss: 0.7041, Test Accuracy: 2390.0000
Epoch 10, Loss: 0.7046, Test Accuracy: 2390.0000
NDCG: 0.5257, Precision: 0.4925, Recall: 0.4928, F1-Score: 0.4878


# GraphSAGE-**amazon**

In [4]:
# Mounting Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Import necessary libraries
import os
import pandas as pd
import numpy as np
import networkx as nx
import torch
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from torch_geometric.nn import SAGEConv  # Use SAGEConv for GraphSAGE
from torch_geometric.data import Data, DataLoader
import torch.optim as optim

# Set environment variables
os.environ['LC_ALL'] = 'en_US.UTF-8'

# Load the Beauty dataset
ratings = pd.read_csv('/content/drive/MyDrive/ratings_Beauty.csv')

# Display the first few rows to check the structure of the dataset
print(ratings.head())

ratings['timestamp'] = pd.to_datetime(ratings['Timestamp'], unit='s')
ratings = ratings.sort_values(by='timestamp')

# Split the data into train and test sets
train_data, test_data = train_test_split(ratings, test_size=0.2, shuffle=False)

train_data = train_data.sample(frac=0.1, random_state=42)
test_data = test_data.sample(frac=0.1, random_state=42)

# Function to create graph from data
def create_graph(data):
    G = nx.DiGraph()
    for _, row in data.iterrows():
        # Use the timestamp() method to get a numeric timestamp
        G.add_edge(row['UserId'], row['ProductId'], timestamp=row['Timestamp'])
    return G

train_graph = create_graph(train_data)
test_graph = create_graph(test_data)

# Function to convert NetworkX graph to PyTorch Geometric Data object
def convert_to_pyg_data(graph, num_features=8):
    nodes = list(graph.nodes())
    node_mapping = {node: i for i, node in enumerate(nodes)}
    edge_index = torch.tensor([[node_mapping[u], node_mapping[v]] for u, v in graph.edges]).t().contiguous()
    edge_time = torch.tensor([graph[u][v]['timestamp'] for u, v in graph.edges], dtype=torch.float)

    # Create a feature matrix with fixed number of features per node
    x = torch.randn(len(nodes), num_features)

    # Random labels for the nodes (binary classification: 0 or 1)
    y = torch.randint(0, 2, (len(nodes),))

    data = Data(x=x, edge_index=edge_index, edge_time=edge_time, y=y)
    return data

train_data_pyg = convert_to_pyg_data(train_graph)
test_data_pyg = convert_to_pyg_data(test_graph)

train_loader = DataLoader([train_data_pyg], batch_size=1, shuffle=True)
test_loader = DataLoader([test_data_pyg], batch_size=1, shuffle=False)

# GraphSAGE Model definition
class GraphSAGE(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GraphSAGE, self).__init__()
        self.conv1 = SAGEConv(in_channels, 8)  # First GraphSAGE layer
        self.conv2 = SAGEConv(8, out_channels)  # Second GraphSAGE layer

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return x

# Initialize the model, loss function, and optimizer
model = GraphSAGE(in_channels=train_data_pyg.num_node_features, out_channels=2)
optimizer = optim.Adam(model.parameters(), lr=0.01)
loss_fn = torch.nn.CrossEntropyLoss()

# Training function
def train(model, loader, optimizer, loss_fn):
    model.train()
    total_loss = 0
    for data in loader:
        optimizer.zero_grad()
        out = model(data.x, data.edge_index)  # GraphSAGE does not use edge_time
        loss = loss_fn(out, data.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

# Evaluation function
def evaluate(model, loader):
    model.eval()
    correct = 0
    for data in loader:
        out = model(data.x, data.edge_index)  # GraphSAGE does not use edge_time
        pred = out.argmax(dim=1)
        correct += (pred == data.y).sum().item()
    return correct / len(loader.dataset)

# Training loop (now running for only 10 epochs)
for epoch in range(10):  # Update: Loop only for 10 epochs
    train_loss = train(model, train_loader, optimizer, loss_fn)
    test_acc = evaluate(model, test_loader)
    print(f'Epoch {epoch+1}, Loss: {train_loss:.4f}, Test Accuracy: {test_acc:.4f}')

# Additional evaluation metrics
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# Function to calculate MRR
def mrr_score(y_true, y_pred):
    order = np.argsort(y_pred)[::-1]
    ranks = np.where(y_true[order] == 1)[0] + 1
    return np.mean(1.0 / ranks)

# Function to calculate NDCG
def ndcg_score(y_true, y_pred, k=10):
    order = np.argsort(y_pred)[::-1]
    y_true = np.take(y_true, order[:k])

    gains = 2 ** y_true - 1
    discounts = np.log2(np.arange(2, k + 2))
    dcg = np.sum(gains / discounts)

    ideal_gains = 2 ** np.sort(y_true)[::-1] - 1
    idcg = np.sum(ideal_gains / discounts)

    return dcg / idcg if idcg > 0 else 0.0

# Evaluation function with metrics
def evaluate_with_metrics(model, loader):
    model.eval()
    all_preds = []
    all_labels = []
    for data in loader:
        out = model(data.x, data.edge_index)  # GraphSAGE does not use edge_time
        pred = out.argmax(dim=1)
        all_preds.append(pred.detach().cpu().numpy())
        all_labels.append(data.y.detach().cpu().numpy())

    all_preds = np.concatenate(all_preds)
    all_labels = np.concatenate(all_labels)

    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='macro')
    recall = recall_score(all_labels, all_preds, average='macro')
    f1 = f1_score(all_labels, all_preds, average='macro')
    mrr = mrr_score(all_labels, all_preds)
    ndcg = ndcg_score(all_labels, all_preds)

    return accuracy, precision, recall, f1, mrr, ndcg

# Final evaluation
accuracy, precision, recall, f1, mrr, ndcg = evaluate_with_metrics(model, test_loader)
print(f'NDCG: {ndcg:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1-Score: {f1:.4f}')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
           UserId   ProductId  Rating   Timestamp
0  A39HTATAQ9V7YF  0205616461     5.0  1369699200
1  A3JM6GV9MNOF9X  0558925278     3.0  1355443200
2  A1Z513UWSAAO0F  0558925278     5.0  1404691200
3  A1WMRR494NWEWV  0733001998     4.0  1382572800
4  A3IAAVS479H7M7  0737104473     1.0  1274227200


/usr/local/lib/python3.11/dist-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Epoch 1, Loss: 0.7067, Test Accuracy: 30706.0000
Epoch 2, Loss: 0.7008, Test Accuracy: 30732.0000
Epoch 3, Loss: 0.6979, Test Accuracy: 30723.0000
Epoch 4, Loss: 0.6970, Test Accuracy: 30714.0000
Epoch 5, Loss: 0.6969, Test Accuracy: 30675.0000
Epoch 6, Loss: 0.6967, Test Accuracy: 30705.0000
Epoch 7, Loss: 0.6961, Test Accuracy: 30708.0000
Epoch 8, Loss: 0.6954, Test Accuracy: 30787.0000
Epoch 9, Loss: 0.6947, Test Accuracy: 30792.0000
Epoch 10, Loss: 0.6942, Test Accuracy: 30836.0000
NDCG: 0.5905, Precision: 0.5021, Recall: 0.5020, F1-Score: 0.4962


# TGN-**houses**

In [40]:
# Mounting Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Import necessary libraries
import os
import pandas as pd
import numpy as np
import networkx as nx
import torch
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from torch_geometric.data import Data, DataLoader
import torch.optim as optim

# Set environment variables
os.environ['LC_ALL'] = 'en_US.UTF-8'

# Load the dataset
ratings = pd.read_csv('/content/drive/MyDrive/user_activity.csv')

# Convert timestamp to datetime and sort by timestamp
ratings['timestamp'] = pd.to_datetime(ratings['create_timestamp'])
ratings = ratings.sort_values(by='timestamp')

# Split the data into train and test sets
train_data, test_data = train_test_split(ratings, test_size=0.2, shuffle=False)

# Function to create graph from data
def create_graph(data):
    G = nx.DiGraph()
    for _, row in data.iterrows():
        # Use the timestamp() method to get a numeric timestamp
        G.add_edge(row['user_id'], row['item_id'], timestamp=row['timestamp'].timestamp())
    return G

train_graph = create_graph(train_data)
test_graph = create_graph(test_data)

# Function to convert NetworkX graph to PyTorch Geometric Data object
def convert_to_pyg_data(graph, num_features=8):
    nodes = list(graph.nodes())
    node_mapping = {node: i for i, node in enumerate(nodes)}
    edge_index = torch.tensor([[node_mapping[u], node_mapping[v]] for u, v in graph.edges]).t().contiguous()
    edge_time = torch.tensor([graph[u][v]['timestamp'] for u, v in graph.edges], dtype=torch.float)

    # Create a feature matrix with fixed number of features per node
    x = torch.randn(len(nodes), num_features)

    # Random labels for the nodes (binary classification: 0 or 1)
    y = torch.randint(0, 2, (len(nodes),))

    data = Data(x=x, edge_index=edge_index, edge_time=edge_time, y=y)
    return data

train_data_pyg = convert_to_pyg_data(train_graph)
test_data_pyg = convert_to_pyg_data(test_graph)

train_loader = DataLoader([train_data_pyg], batch_size=1, shuffle=True)
test_loader = DataLoader([test_data_pyg], batch_size=1, shuffle=False)

# Custom TGN Model definition
class TGNModel(torch.nn.Module):
    def __init__(self, in_channels, out_channels, memory_dim=8, time_dim=8):
        super(TGNModel, self).__init__()
        self.memory_dim = memory_dim
        self.time_dim = time_dim

        # Memory for each node
        self.memory = torch.zeros(train_data_pyg.num_nodes, memory_dim)

        # Embedding for time
        self.time_embedding = torch.nn.Embedding(365, time_dim)

        # Message and memory update functions
        self.message_fn = torch.nn.Linear(in_channels + memory_dim + time_dim, memory_dim)
        self.memory_update_fn = torch.nn.GRUCell(memory_dim, memory_dim)

        # Final classification layer
        self.fc = torch.nn.Linear(memory_dim, out_channels)

    def forward(self, x, edge_index, edge_time):
        # Get memory embeddings for source and target nodes
        src, dst = edge_index
        src_memory = self.memory[src]
        dst_memory = self.memory[dst]

        # Embed time
        time_embeds = self.time_embedding((edge_time.long() % 365).view(-1, 1)).view(-1, self.time_dim)

        # Create messages
        messages = self.message_fn(torch.cat([x[src], src_memory, time_embeds], dim=1))

        # Update memory for destination nodes (avoid in-place updates)
        updated_memory = self.memory_update_fn(messages, dst_memory)
        self.memory[dst] = updated_memory.detach()  # Detach to avoid breaking the computation graph

        # Apply final classification layer
        out = self.fc(updated_memory)  # Only output predictions for destination nodes
        return out

# Initialize the model, loss function, and optimizer
model = TGNModel(in_channels=train_data_pyg.num_node_features, out_channels=2)
optimizer = optim.Adam(model.parameters(), lr=0.01)
loss_fn = torch.nn.CrossEntropyLoss()

# Training function
def train(model, loader, optimizer, loss_fn):
    model.train()
    total_loss = 0
    for data in loader:
        optimizer.zero_grad()
        out = model(data.x, data.edge_index, data.edge_time)  # TGN uses edge_time
        loss = loss_fn(out, data.y[data.edge_index[1]])  # Only compute loss for destination nodes
        loss.backward(retain_graph=True)  # Retain the computation graph
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

# Evaluation function
def evaluate(model, loader):
    model.eval()
    correct = 0
    total = 0
    for data in loader:
        out = model(data.x, data.edge_index, data.edge_time)  # TGN uses edge_time
        pred = out.argmax(dim=1)
        correct += (pred == data.y[data.edge_index[1]]).sum().item()  # Only evaluate destination nodes
        total += len(data.edge_index[1])
    return correct / total

# Training loop (now running for only 10 epochs)
for epoch in range(10):  # Update: Loop only for 10 epochs
    train_loss = train(model, train_loader, optimizer, loss_fn)
    test_acc = evaluate(model, test_loader)
    print(f'Epoch {epoch+1}, Loss: {train_loss:.4f}, Test Accuracy: {test_acc:.4f}')

# Additional evaluation metrics
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# Function to calculate MRR
def mrr_score(y_true, y_pred):
    order = np.argsort(y_pred)[::-1]
    ranks = np.where(y_true[order] == 1)[0] + 1
    return np.mean(1.0 / ranks)

# Function to calculate NDCG
def ndcg_score(y_true, y_pred, k=10):
    order = np.argsort(y_pred)[::-1]
    y_true = np.take(y_true, order[:k])

    gains = 2 ** y_true - 1
    discounts = np.log2(np.arange(2, k + 2))
    dcg = np.sum(gains / discounts)

    ideal_gains = 2 ** np.sort(y_true)[::-1] - 1
    idcg = np.sum(ideal_gains / discounts)

    return dcg / idcg if idcg > 0 else 0.0

# Evaluation function with metrics
def evaluate_with_metrics(model, loader):
    model.eval()
    all_preds = []
    all_labels = []
    for data in loader:
        out = model(data.x, data.edge_index, data.edge_time)  # TGN uses edge_time
        pred = out.argmax(dim=1)
        all_preds.append(pred.detach().cpu().numpy())
        all_labels.append(data.y[data.edge_index[1]].detach().cpu().numpy())  # Only evaluate destination nodes

    all_preds = np.concatenate(all_preds)
    all_labels = np.concatenate(all_labels)

    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='macro')
    recall = recall_score(all_labels, all_preds, average='macro')
    f1 = f1_score(all_labels, all_preds, average='macro')
    mrr = mrr_score(all_labels, all_preds)
    ndcg = ndcg_score(all_labels, all_preds)

    return accuracy, precision, recall, f1, mrr, ndcg

# Final evaluation
accuracy, precision, recall, f1, mrr, ndcg = evaluate_with_metrics(model, test_loader)
print(f'NDCG: {ndcg:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1-Score: {f1:.4f}')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.11/dist-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Epoch 1, Loss: 0.6937, Test Accuracy: 0.4981
Epoch 2, Loss: 0.6930, Test Accuracy: 0.4966
Epoch 3, Loss: 0.6933, Test Accuracy: 0.4968
Epoch 4, Loss: 0.6934, Test Accuracy: 0.4967
Epoch 5, Loss: 0.6930, Test Accuracy: 0.4962
Epoch 6, Loss: 0.6925, Test Accuracy: 0.4946
Epoch 7, Loss: 0.6923, Test Accuracy: 0.4930
Epoch 8, Loss: 0.6922, Test Accuracy: 0.4931
Epoch 9, Loss: 0.6919, Test Accuracy: 0.4952
Epoch 10, Loss: 0.6917, Test Accuracy: 0.4959
NDCG: 0.5261, Precision: 0.4909, Recall: 0.4990, F1-Score: 0.3537


# TGN-**movies**

In [49]:
# Mounting Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Import necessary libraries
import os
import pandas as pd
import numpy as np
import networkx as nx
import torch
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from torch_geometric.data import Data, DataLoader
import torch.optim as optim

# Set environment variables
os.environ['LC_ALL'] = 'en_US.UTF-8'

# Load the dataset (using ratings_small.csv from MovieLens)
ratings = pd.read_csv('/content/drive/MyDrive/movielens/ratings_small.csv')

# Convert timestamp to datetime and sort by timestamp
ratings['timestamp'] = pd.to_datetime(ratings['timestamp'])
ratings = ratings.sort_values(by='timestamp')

# Split the data into train and test sets
train_data, test_data = train_test_split(ratings, test_size=0.2, shuffle=False)

# Function to create graph from data
def create_graph(data):
    G = nx.DiGraph()
    for _, row in data.iterrows():
        # Use the timestamp() method to get a numeric timestamp
        G.add_edge(row['userId'], row['movieId'], timestamp=row['timestamp'].timestamp())
    return G

train_graph = create_graph(train_data)
test_graph = create_graph(test_data)

# Function to convert NetworkX graph to PyTorch Geometric Data object
def convert_to_pyg_data(graph, num_features=8):
    nodes = list(graph.nodes())
    node_mapping = {node: i for i, node in enumerate(nodes)}
    edge_index = torch.tensor([[node_mapping[u], node_mapping[v]] for u, v in graph.edges]).t().contiguous()
    edge_time = torch.tensor([graph[u][v]['timestamp'] for u, v in graph.edges], dtype=torch.float)

    # Create a feature matrix with fixed number of features per node
    x = torch.randn(len(nodes), num_features)

    # Random labels for the nodes (binary classification: 0 or 1)
    y = torch.randint(0, 2, (len(nodes),))

    data = Data(x=x, edge_index=edge_index, edge_time=edge_time, y=y)
    return data

train_data_pyg = convert_to_pyg_data(train_graph)
test_data_pyg = convert_to_pyg_data(test_graph)

train_loader = DataLoader([train_data_pyg], batch_size=1, shuffle=True)
test_loader = DataLoader([test_data_pyg], batch_size=1, shuffle=False)

# Custom TGN Model definition
class TGNModel(torch.nn.Module):
    def __init__(self, in_channels, out_channels, memory_dim=8, time_dim=8):
        super(TGNModel, self).__init__()
        self.memory_dim = memory_dim
        self.time_dim = time_dim

        # Memory for each node
        self.memory = torch.zeros(train_data_pyg.num_nodes, memory_dim)

        # Embedding for time
        self.time_embedding = torch.nn.Embedding(365, time_dim)

        # Message and memory update functions
        self.message_fn = torch.nn.Linear(in_channels + memory_dim + time_dim, memory_dim)
        self.memory_update_fn = torch.nn.GRUCell(memory_dim, memory_dim)

        # Final classification layer
        self.fc = torch.nn.Linear(memory_dim, out_channels)

    def forward(self, x, edge_index, edge_time):
        # Get memory embeddings for source and target nodes
        src, dst = edge_index
        src_memory = self.memory[src]
        dst_memory = self.memory[dst]

        # Embed time
        time_embeds = self.time_embedding((edge_time.long() % 365).view(-1, 1)).view(-1, self.time_dim)

        # Create messages
        messages = self.message_fn(torch.cat([x[src], src_memory, time_embeds], dim=1))

        # Update memory for destination nodes (avoid in-place updates)
        updated_memory = self.memory_update_fn(messages, dst_memory)
        self.memory[dst] = updated_memory.detach()  # Detach to avoid breaking the computation graph

        # Apply final classification layer
        out = self.fc(updated_memory)  # Only output predictions for destination nodes
        return out

# Initialize the model, loss function, and optimizer
model = TGNModel(in_channels=train_data_pyg.num_node_features, out_channels=2)
optimizer = optim.Adam(model.parameters(), lr=0.01)
loss_fn = torch.nn.CrossEntropyLoss()

# Training function
def train(model, loader, optimizer, loss_fn):
    model.train()
    total_loss = 0
    for data in loader:
        optimizer.zero_grad()
        out = model(data.x, data.edge_index, data.edge_time)  # TGN uses edge_time
        loss = loss_fn(out, data.y[data.edge_index[1]])  # Only compute loss for destination nodes
        loss.backward(retain_graph=True)  # Retain the computation graph
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

# Evaluation function
def evaluate(model, loader):
    model.eval()
    correct = 0
    total = 0
    for data in loader:
        out = model(data.x, data.edge_index, data.edge_time)  # TGN uses edge_time
        pred = out.argmax(dim=1)
        correct += (pred == data.y[data.edge_index[1]]).sum().item()  # Only evaluate destination nodes
        total += len(data.edge_index[1])
    return correct / total

# Training loop (now running for only 10 epochs)
for epoch in range(10):  # Update: Loop only for 10 epochs
    train_loss = train(model, train_loader, optimizer, loss_fn)
    test_acc = evaluate(model, test_loader)
    print(f'Epoch {epoch+1}, Loss: {train_loss:.4f}, Test Accuracy: {test_acc:.4f}')

# Additional evaluation metrics
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# Function to calculate MRR
def mrr_score(y_true, y_pred):
    order = np.argsort(y_pred)[::-1]
    ranks = np.where(y_true[order] == 1)[0] + 1
    return np.mean(1.0 / ranks)

# Function to calculate NDCG
def ndcg_score(y_true, y_pred, k=10):
    order = np.argsort(y_pred)[::-1]
    y_true = np.take(y_true, order[:k])

    gains = 2 ** y_true - 1
    discounts = np.log2(np.arange(2, k + 2))
    dcg = np.sum(gains / discounts)

    ideal_gains = 2 ** np.sort(y_true)[::-1] - 1
    idcg = np.sum(ideal_gains / discounts)

    return dcg / idcg if idcg > 0 else 0.0

# Evaluation function with metrics
def evaluate_with_metrics(model, loader):
    model.eval()
    all_preds = []
    all_labels = []
    for data in loader:
        out = model(data.x, data.edge_index, data.edge_time)  # TGN uses edge_time
        pred = out.argmax(dim=1)
        all_preds.append(pred.detach().cpu().numpy())
        all_labels.append(data.y[data.edge_index[1]].detach().cpu().numpy())  # Only evaluate destination nodes

    all_preds = np.concatenate(all_preds)
    all_labels = np.concatenate(all_labels)

    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='macro')
    recall = recall_score(all_labels, all_preds, average='macro')
    f1 = f1_score(all_labels, all_preds, average='macro')
    mrr = mrr_score(all_labels, all_preds)
    ndcg = ndcg_score(all_labels, all_preds)

    return accuracy, precision, recall, f1, mrr, ndcg

# Final evaluation
accuracy, precision, recall, f1, mrr, ndcg = evaluate_with_metrics(model, test_loader)
print(f'NDCG: {ndcg:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1-Score: {f1:.4f}')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.11/dist-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Epoch 1, Loss: 0.6965, Test Accuracy: 0.4896
Epoch 2, Loss: 0.6950, Test Accuracy: 0.5115
Epoch 3, Loss: 0.6943, Test Accuracy: 0.5161
Epoch 4, Loss: 0.6957, Test Accuracy: 0.5163
Epoch 5, Loss: 0.6953, Test Accuracy: 0.5163
Epoch 6, Loss: 0.6938, Test Accuracy: 0.5161
Epoch 7, Loss: 0.6929, Test Accuracy: 0.5087
Epoch 8, Loss: 0.6930, Test Accuracy: 0.4982
Epoch 9, Loss: 0.6936, Test Accuracy: 0.4957
Epoch 10, Loss: 0.6938, Test Accuracy: 0.4982
NDCG: 0.6031, Precision: 0.5020, Recall: 0.5020, F1-Score: 0.4952


# TGN-**amazon**

In [6]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Import necessary libraries
import os
import pandas as pd
import numpy as np
import networkx as nx
import torch
import torch.optim as optim
from sklearn.model_selection import train_test_split
from torch_geometric.data import Data, DataLoader
import torch.nn.functional as F

# Set environment variables
os.environ['LC_ALL'] = 'en_US.UTF-8'

# Load the Beauty dataset
ratings = pd.read_csv('/content/drive/MyDrive/ratings_Beauty.csv')

# Display the first few rows to check the structure of the dataset
print(ratings.head())

ratings['timestamp'] = pd.to_datetime(ratings['Timestamp'], unit='s')
ratings = ratings.sort_values(by='timestamp')

# Split the data into train and test sets
train_data, test_data = train_test_split(ratings, test_size=0.2, shuffle=False)

train_data = train_data.sample(frac=0.1, random_state=42)
test_data = test_data.sample(frac=0.1, random_state=42)

# Function to create graph from data
def create_graph(data):
    G = nx.DiGraph()
    for _, row in data.iterrows():
         G.add_edge(row['UserId'], row['ProductId'], timestamp=row['Timestamp'])
    return G

train_graph = create_graph(train_data)
test_graph = create_graph(test_data)

# Function to convert NetworkX graph to PyTorch Geometric Data object
def convert_to_pyg_data(graph, num_features=8):
    nodes = list(graph.nodes())
    node_mapping = {node: i for i, node in enumerate(nodes)}
    edge_index = torch.tensor([[node_mapping[u], node_mapping[v]] for u, v in graph.edges]).t().contiguous()
    edge_time = torch.tensor([graph[u][v]['timestamp'] for u, v in graph.edges], dtype=torch.float)

    # Create a feature matrix with fixed number of features per node
    x = torch.randn(len(nodes), num_features)

    # Random labels for the nodes (binary classification: 0 or 1)
    y = torch.randint(0, 2, (len(nodes),))

    data = Data(x=x, edge_index=edge_index, edge_time=edge_time, y=y)
    return data

train_data_pyg = convert_to_pyg_data(train_graph)
test_data_pyg = convert_to_pyg_data(test_graph)

train_loader = DataLoader([train_data_pyg], batch_size=1, shuffle=True)
test_loader = DataLoader([test_data_pyg], batch_size=1, shuffle=False)

# Custom TGN Model definition
class TGNModel(torch.nn.Module):
    def __init__(self, in_channels, out_channels, memory_dim=8, time_dim=8):
        super(TGNModel, self).__init__()
        self.memory_dim = memory_dim
        self.time_dim = time_dim

        # Memory for each node
        self.memory = torch.zeros(train_data_pyg.num_nodes, memory_dim)

        # Embedding for time
        self.time_embedding = torch.nn.Embedding(365, time_dim)

        # Message and memory update functions
        self.message_fn = torch.nn.Linear(in_channels + memory_dim + time_dim, memory_dim)
        self.memory_update_fn = torch.nn.GRUCell(memory_dim, memory_dim)

        # Final classification layer
        self.fc = torch.nn.Linear(memory_dim, out_channels)

    def forward(self, x, edge_index, edge_time):
        # Get memory embeddings for source and target nodes
        src, dst = edge_index
        src_memory = self.memory[src]
        dst_memory = self.memory[dst]

        # Embed time
        time_embeds = self.time_embedding((edge_time.long() % 365).view(-1, 1)).view(-1, self.time_dim)

        # Create messages
        messages = self.message_fn(torch.cat([x[src], src_memory, time_embeds], dim=1))

        # Update memory for destination nodes (avoid in-place updates)
        updated_memory = self.memory_update_fn(messages, dst_memory)
        self.memory[dst] = updated_memory.detach()  # Detach to avoid breaking the computation graph

        # Apply final classification layer
        out = self.fc(updated_memory)  # Only output predictions for destination nodes
        return out

# Initialize the model, loss function, and optimizer
model = TGNModel(in_channels=train_data_pyg.num_node_features, out_channels=2)
optimizer = optim.Adam(model.parameters(), lr=0.01)
loss_fn = torch.nn.CrossEntropyLoss()

# Training function
def train(model, loader, optimizer, loss_fn):
    model.train()
    total_loss = 0
    for data in loader:
        optimizer.zero_grad()
        out = model(data.x, data.edge_index, data.edge_time)  # TGN uses edge_time
        loss = loss_fn(out, data.y[data.edge_index[1]])  # Only compute loss for destination nodes
        loss.backward(retain_graph=True)  # Retain the computation graph
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

# Evaluation function
def evaluate(model, loader):
    model.eval()
    correct = 0
    total = 0
    for data in loader:
        out = model(data.x, data.edge_index, data.edge_time)  # TGN uses edge_time
        pred = out.argmax(dim=1)
        correct += (pred == data.y[data.edge_index[1]]).sum().item()  # Only evaluate destination nodes
        total += len(data.edge_index[1])
    return correct / total

# Training loop (now running for only 10 epochs)
for epoch in range(10):  # Update: Loop only for 10 epochs
    train_loss = train(model, train_loader, optimizer, loss_fn)
    test_acc = evaluate(model, test_loader)
    print(f'Epoch {epoch+1}, Loss: {train_loss:.4f}, Test Accuracy: {test_acc:.4f}')

# Additional evaluation metrics
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# Function to calculate MRR
def mrr_score(y_true, y_pred):
    order = np.argsort(y_pred)[::-1]
    ranks = np.where(y_true[order] == 1)[0] + 1
    return np.mean(1.0 / ranks)

# Function to calculate NDCG
def ndcg_score(y_true, y_pred, k=10):
    order = np.argsort(y_pred)[::-1]
    y_true = np.take(y_true, order[:k])

    gains = 2 ** y_true - 1
    discounts = np.log2(np.arange(2, k + 2))
    dcg = np.sum(gains / discounts)

    ideal_gains = 2 ** np.sort(y_true)[::-1] - 1
    idcg = np.sum(ideal_gains / discounts)

    return dcg / idcg if idcg > 0 else 0.0

# Evaluation function with metrics
def evaluate_with_metrics(model, loader):
    model.eval()
    all_preds = []
    all_labels = []
    for data in loader:
        out = model(data.x, data.edge_index, data.edge_time)  # TGN uses edge_time
        pred = out.argmax(dim=1)
        all_preds.append(pred.detach().cpu().numpy())
        all_labels.append(data.y[data.edge_index[1]].detach().cpu().numpy())  # Only evaluate destination nodes

    all_preds = np.concatenate(all_preds)
    all_labels = np.concatenate(all_labels)

    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='macro')
    recall = recall_score(all_labels, all_preds, average='macro')
    f1 = f1_score(all_labels, all_preds, average='macro')
    mrr = mrr_score(all_labels, all_preds)
    ndcg = ndcg_score(all_labels, all_preds)

    return accuracy, precision, recall, f1, mrr, ndcg

# Final evaluation
accuracy, precision, recall, f1, mrr, ndcg = evaluate_with_metrics(model, test_loader)
print(f'NDCG: {ndcg:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1-Score: {f1:.4f}')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
           UserId   ProductId  Rating   Timestamp
0  A39HTATAQ9V7YF  0205616461     5.0  1369699200
1  A3JM6GV9MNOF9X  0558925278     3.0  1355443200
2  A1Z513UWSAAO0F  0558925278     5.0  1404691200
3  A1WMRR494NWEWV  0733001998     4.0  1382572800
4  A3IAAVS479H7M7  0737104473     1.0  1274227200


/usr/local/lib/python3.11/dist-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Epoch 1, Loss: 0.7183, Test Accuracy: 0.4933
Epoch 2, Loss: 0.7154, Test Accuracy: 0.4938
Epoch 3, Loss: 0.7077, Test Accuracy: 0.4946
Epoch 4, Loss: 0.7012, Test Accuracy: 0.4989
Epoch 5, Loss: 0.6970, Test Accuracy: 0.5008
Epoch 6, Loss: 0.6950, Test Accuracy: 0.5029
Epoch 7, Loss: 0.6945, Test Accuracy: 0.5024
Epoch 8, Loss: 0.6947, Test Accuracy: 0.5056
Epoch 9, Loss: 0.6952, Test Accuracy: 0.5064
Epoch 10, Loss: 0.6955, Test Accuracy: 0.5074
NDCG: 0.4401, Precision: 0.5036, Recall: 0.5017, F1-Score: 0.4298


# RNN-**houses**

In [44]:
# Mounting Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Import necessary libraries
import os
import pandas as pd
import numpy as np
import networkx as nx
import torch
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from torch_geometric.data import Data, DataLoader
import torch.optim as optim
from torch.nn import RNN, Linear

# Set environment variables
os.environ['LC_ALL'] = 'en_US.UTF-8'

# Load the dataset
ratings = pd.read_csv('/content/drive/MyDrive/user_activity.csv')

# Convert timestamp to datetime and sort by timestamp
ratings['timestamp'] = pd.to_datetime(ratings['create_timestamp'])
ratings = ratings.sort_values(by='timestamp')

# Split the data into train and test sets
train_data, test_data = train_test_split(ratings, test_size=0.2, shuffle=False)

# Function to create graph from data
def create_graph(data):
    G = nx.DiGraph()
    for _, row in data.iterrows():
        # Use the timestamp() method to get a numeric timestamp
        G.add_edge(row['user_id'], row['item_id'], timestamp=row['timestamp'].timestamp())
    return G

train_graph = create_graph(train_data)
test_graph = create_graph(test_data)

# Function to convert NetworkX graph to PyTorch Geometric Data object
def convert_to_pyg_data(graph, num_features=8):
    nodes = list(graph.nodes())
    node_mapping = {node: i for i, node in enumerate(nodes)}
    edge_index = torch.tensor([[node_mapping[u], node_mapping[v]] for u, v in graph.edges]).t().contiguous()
    edge_time = torch.tensor([graph[u][v]['timestamp'] for u, v in graph.edges], dtype=torch.float)

    # Create a feature matrix with fixed number of features per node
    x = torch.randn(len(nodes), num_features)

    # Random labels for the nodes (binary classification: 0 or 1)
    y = torch.randint(0, 2, (len(nodes),))

    data = Data(x=x, edge_index=edge_index, edge_time=edge_time, y=y)
    return data

train_data_pyg = convert_to_pyg_data(train_graph)
test_data_pyg = convert_to_pyg_data(test_graph)

train_loader = DataLoader([train_data_pyg], batch_size=1, shuffle=True)
test_loader = DataLoader([test_data_pyg], batch_size=1, shuffle=False)

# RNN Model definition
class RNNModel(torch.nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNNModel, self).__init__()
        self.hidden_size = hidden_size
        self.rnn = RNN(input_size, hidden_size, batch_first=True)
        self.fc = Linear(hidden_size, output_size)

    def forward(self, x, edge_index, edge_time):
        # Prepare input for RNN
        # x: Node features (num_nodes, num_features)
        # edge_index: Edge connections (2, num_edges)
        # edge_time: Timestamps for edges (num_edges,)

        # Sort edges by timestamp
        sorted_indices = torch.argsort(edge_time)
        sorted_edge_index = edge_index[:, sorted_indices]
        sorted_edge_time = edge_time[sorted_indices]

        # Prepare sequences for RNN
        src, dst = sorted_edge_index
        sequences = x[src]  # Use source node features as input sequences

        # Initialize hidden state with correct batch size
        batch_size = sequences.size(0)  # Number of edges
        h0 = torch.zeros(1, batch_size, self.hidden_size)  # (num_layers, batch_size, hidden_size)

        # Pass sequences through RNN
        out, _ = self.rnn(sequences.unsqueeze(1), h0)  # Add sequence length dimension
        out = out.squeeze(1)  # Remove sequence length dimension

        # Apply final classification layer
        out = self.fc(out)
        return out  # Output predictions for all edges

# Initialize the model, loss function, and optimizer
input_size = train_data_pyg.num_node_features
hidden_size = 16
output_size = 2
model = RNNModel(input_size, hidden_size, output_size)
optimizer = optim.Adam(model.parameters(), lr=0.01)
loss_fn = torch.nn.CrossEntropyLoss()

# Training function
def train(model, loader, optimizer, loss_fn):
    model.train()
    total_loss = 0
    for data in loader:
        optimizer.zero_grad()
        out = model(data.x, data.edge_index, data.edge_time)  # RNN uses edge_time
        loss = loss_fn(out, data.y[data.edge_index[1]])  # Only compute loss for destination nodes
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

# Evaluation function
def evaluate(model, loader):
    model.eval()
    correct = 0
    total = 0
    for data in loader:
        out = model(data.x, data.edge_index, data.edge_time)  # RNN uses edge_time
        pred = out.argmax(dim=1)
        correct += (pred == data.y[data.edge_index[1]]).sum().item()  # Only evaluate destination nodes
        total += len(data.edge_index[1])
    return correct / total

# Training loop (now running for only 10 epochs)
for epoch in range(10):  # Update: Loop only for 10 epochs
    train_loss = train(model, train_loader, optimizer, loss_fn)
    test_acc = evaluate(model, test_loader)
    print(f'Epoch {epoch+1}, Loss: {train_loss:.4f}, Test Accuracy: {test_acc:.4f}')

# Additional evaluation metrics
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# Function to calculate MRR
def mrr_score(y_true, y_pred):
    order = np.argsort(y_pred)[::-1]
    ranks = np.where(y_true[order] == 1)[0] + 1
    return np.mean(1.0 / ranks)

# Function to calculate NDCG
def ndcg_score(y_true, y_pred, k=10):
    order = np.argsort(y_pred)[::-1]
    y_true = np.take(y_true, order[:k])

    gains = 2 ** y_true - 1
    discounts = np.log2(np.arange(2, k + 2))
    dcg = np.sum(gains / discounts)

    ideal_gains = 2 ** np.sort(y_true)[::-1] - 1
    idcg = np.sum(ideal_gains / discounts)

    return dcg / idcg if idcg > 0 else 0.0

# Evaluation function with metrics
def evaluate_with_metrics(model, loader):
    model.eval()
    all_preds = []
    all_labels = []
    for data in loader:
        out = model(data.x, data.edge_index, data.edge_time)  # RNN uses edge_time
        pred = out.argmax(dim=1)
        all_preds.append(pred.detach().cpu().numpy())
        all_labels.append(data.y[data.edge_index[1]].detach().cpu().numpy())  # Only evaluate destination nodes

    all_preds = np.concatenate(all_preds)
    all_labels = np.concatenate(all_labels)

    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='macro')
    recall = recall_score(all_labels, all_preds, average='macro')
    f1 = f1_score(all_labels, all_preds, average='macro')
    mrr = mrr_score(all_labels, all_preds)
    ndcg = ndcg_score(all_labels, all_preds)

    return accuracy, precision, recall, f1, mrr, ndcg

# Final evaluation
accuracy, precision, recall, f1, mrr, ndcg = evaluate_with_metrics(model, test_loader)
print(f'NDCG: {ndcg:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1-Score: {f1:.4f}')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.11/dist-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Epoch 1, Loss: 0.7266, Test Accuracy: 0.4842
Epoch 2, Loss: 0.7121, Test Accuracy: 0.4865
Epoch 3, Loss: 0.7025, Test Accuracy: 0.4959
Epoch 4, Loss: 0.6971, Test Accuracy: 0.5074
Epoch 5, Loss: 0.6949, Test Accuracy: 0.5232
Epoch 6, Loss: 0.6947, Test Accuracy: 0.5255
Epoch 7, Loss: 0.6952, Test Accuracy: 0.5254
Epoch 8, Loss: 0.6955, Test Accuracy: 0.5240
Epoch 9, Loss: 0.6954, Test Accuracy: 0.5214
Epoch 10, Loss: 0.6952, Test Accuracy: 0.5181
NDCG: 0.4790, Precision: 0.5031, Recall: 0.5019, F1-Score: 0.4582


# RNN-**movies**

In [52]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Import necessary libraries
import os
import pandas as pd
import numpy as np
import networkx as nx
import torch
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from torch_geometric.data import Data, DataLoader
import torch.optim as optim
from torch.nn import RNN, Linear

# Set environment variables
os.environ['LC_ALL'] = 'en_US.UTF-8'

# Load the dataset
ratings = pd.read_csv('/content/drive/MyDrive/movielens/ratings_small.csv')

# Convert timestamp to datetime and sort by timestamp
ratings['timestamp'] = pd.to_datetime(ratings['timestamp'])
ratings = ratings.sort_values(by='timestamp')

# Split the data into train and test sets
train_data, test_data = train_test_split(ratings, test_size=0.2, shuffle=False)

# Function to create graph from data
def create_graph(data):
    G = nx.DiGraph()
    for _, row in data.iterrows():
        G.add_edge(row['userId'], row['movieId'], timestamp=row['timestamp'].timestamp())
    return G

train_graph = create_graph(train_data)
test_graph = create_graph(test_data)

# Function to convert NetworkX graph to PyTorch Geometric Data object
def convert_to_pyg_data(graph, num_features=8):
    nodes = list(graph.nodes())
    node_mapping = {node: i for i, node in enumerate(nodes)}
    edge_index = torch.tensor([[node_mapping[u], node_mapping[v]] for u, v in graph.edges]).t().contiguous()
    edge_time = torch.tensor([graph[u][v]['timestamp'] for u, v in graph.edges], dtype=torch.float)

    # Create a feature matrix with fixed number of features per node
    x = torch.randn(len(nodes), num_features)

    # Random labels for the nodes (binary classification: 0 or 1)
    y = torch.randint(0, 2, (len(nodes),))

    data = Data(x=x, edge_index=edge_index, edge_time=edge_time, y=y)
    return data

train_data_pyg = convert_to_pyg_data(train_graph)
test_data_pyg = convert_to_pyg_data(test_graph)

train_loader = DataLoader([train_data_pyg], batch_size=1, shuffle=True)
test_loader = DataLoader([test_data_pyg], batch_size=1, shuffle=False)

# RNN Model definition
class RNNModel(torch.nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNNModel, self).__init__()
        self.hidden_size = hidden_size
        self.rnn = RNN(input_size, hidden_size, batch_first=True)
        self.fc = Linear(hidden_size, output_size)

    def forward(self, x, edge_index, edge_time):
        # Sort edges by timestamp
        sorted_indices = torch.argsort(edge_time)
        sorted_edge_index = edge_index[:, sorted_indices]
        sorted_edge_time = edge_time[sorted_indices]

        # Prepare sequences for RNN
        src, dst = sorted_edge_index
        sequences = x[src]  # Use source node features as input sequences

        # Initialize hidden state with correct batch size
        batch_size = sequences.size(0)  # Number of edges
        h0 = torch.zeros(1, batch_size, self.hidden_size)  # (num_layers, batch_size, hidden_size)

        # Pass sequences through RNN
        out, _ = self.rnn(sequences.unsqueeze(1), h0)  # Add sequence length dimension
        out = out.squeeze(1)  # Remove sequence length dimension

        # Apply final classification layer
        out = self.fc(out)
        return out  # Output predictions for all edges

# Initialize the model, loss function, and optimizer
input_size = train_data_pyg.num_node_features
hidden_size = 16
output_size = 2
model = RNNModel(input_size, hidden_size, output_size)
optimizer = optim.Adam(model.parameters(), lr=0.01)
loss_fn = torch.nn.CrossEntropyLoss()

# Training function
def train(model, loader, optimizer, loss_fn):
    model.train()
    total_loss = 0
    for data in loader:
        optimizer.zero_grad()
        out = model(data.x, data.edge_index, data.edge_time)  # RNN uses edge_time
        loss = loss_fn(out, data.y[data.edge_index[1]])  # Only compute loss for destination nodes
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

# Evaluation function
def evaluate(model, loader):
    model.eval()
    correct = 0
    total = 0
    for data in loader:
        out = model(data.x, data.edge_index, data.edge_time)  # RNN uses edge_time
        pred = out.argmax(dim=1)
        correct += (pred == data.y[data.edge_index[1]]).sum().item()  # Only evaluate destination nodes
        total += len(data.edge_index[1])
    return correct / total

# Training loop (now running for only 10 epochs)
for epoch in range(10):  # Update: Loop only for 10 epochs
    train_loss = train(model, train_loader, optimizer, loss_fn)
    test_acc = evaluate(model, test_loader)
    print(f'Epoch {epoch+1}, Loss: {train_loss:.4f}, Test Accuracy: {test_acc:.4f}')

# Additional evaluation metrics
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# Function to calculate MRR
def mrr_score(y_true, y_pred):
    order = np.argsort(y_pred)[::-1]
    ranks = np.where(y_true[order] == 1)[0] + 1
    return np.mean(1.0 / ranks)

# Function to calculate NDCG
def ndcg_score(y_true, y_pred, k=10):
    order = np.argsort(y_pred)[::-1]
    y_true = np.take(y_true, order[:k])

    gains = 2 ** y_true - 1
    discounts = np.log2(np.arange(2, k + 2))
    dcg = np.sum(gains / discounts)

    ideal_gains = 2 ** np.sort(y_true)[::-1] - 1
    idcg = np.sum(ideal_gains / discounts)

    return dcg / idcg if idcg > 0 else 0.0

# Evaluation function with metrics
def evaluate_with_metrics(model, loader):
    model.eval()
    all_preds = []
    all_labels = []
    for data in loader:
        out = model(data.x, data.edge_index, data.edge_time)  # RNN uses edge_time
        pred = out.argmax(dim=1)
        all_preds.append(pred.detach().cpu().numpy())
        all_labels.append(data.y[data.edge_index[1]].detach().cpu().numpy())  # Only evaluate destination nodes

    all_preds = np.concatenate(all_preds)
    all_labels = np.concatenate(all_labels)

    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='macro')
    recall = recall_score(all_labels, all_preds, average='macro')
    f1 = f1_score(all_labels, all_preds, average='macro')
    mrr = mrr_score(all_labels, all_preds)
    ndcg = ndcg_score(all_labels, all_preds)

    return accuracy, precision, recall, f1, mrr, ndcg

# Final evaluation
accuracy, precision, recall, f1, mrr, ndcg = evaluate_with_metrics(model, test_loader)
print(f'NDCG: {ndcg:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1-Score: {f1:.4f}')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.11/dist-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Epoch 1, Loss: 0.7251, Test Accuracy: 0.5158
Epoch 2, Loss: 0.7096, Test Accuracy: 0.5149
Epoch 3, Loss: 0.6995, Test Accuracy: 0.5021
Epoch 4, Loss: 0.6947, Test Accuracy: 0.4916
Epoch 5, Loss: 0.6940, Test Accuracy: 0.4900
Epoch 6, Loss: 0.6957, Test Accuracy: 0.4861
Epoch 7, Loss: 0.6976, Test Accuracy: 0.4854
Epoch 8, Loss: 0.6986, Test Accuracy: 0.4845
Epoch 9, Loss: 0.6983, Test Accuracy: 0.4845
Epoch 10, Loss: 0.6971, Test Accuracy: 0.4842
NDCG: 0.5668, Precision: 0.4900, Recall: 0.4992, F1-Score: 0.3418


# RNN-**amazon**

In [8]:
# Mounting Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Import necessary libraries
import os
import pandas as pd
import networkx as nx
import torch
import torch.optim as optim
from sklearn.model_selection import train_test_split
from torch_geometric.data import Data, DataLoader
import torch.nn as nn
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# Set environment variables
os.environ['LC_ALL'] = 'en_US.UTF-8'

# Load the Beauty dataset
ratings = pd.read_csv('/content/drive/MyDrive/ratings_Beauty.csv')

# Display the first few rows to check the structure of the dataset
print(ratings.head())

ratings['timestamp'] = pd.to_datetime(ratings['Timestamp'], unit='s')
ratings = ratings.sort_values(by='timestamp')

# Split the data into train and test sets
train_data, test_data = train_test_split(ratings, test_size=0.2, shuffle=False)

train_data = train_data.sample(frac=0.1, random_state=42)
test_data = test_data.sample(frac=0.1, random_state=42)

# Function to create graph from data
def create_graph(data):
    G = nx.DiGraph()
    for _, row in data.iterrows():
        # Use the timestamp() method to get a numeric timestamp
        G.add_edge(row['UserId'], row['ProductId'], timestamp=row['Timestamp'])
    return G

# Create graphs for train and test data
train_graph = create_graph(train_data)
test_graph = create_graph(test_data)

# Function to convert NetworkX graph to PyTorch Geometric Data object
def convert_to_pyg_data(graph, num_features=8):
    nodes = list(graph.nodes())
    node_mapping = {node: i for i, node in enumerate(nodes)}
    edge_index = torch.tensor([[node_mapping[u], node_mapping[v]] for u, v in graph.edges]).t().contiguous()
    edge_time = torch.tensor([graph[u][v]['timestamp'] for u, v in graph.edges], dtype=torch.float)

    # Create a feature matrix with fixed number of features per node
    x = torch.randn(len(nodes), num_features)

    # Random labels for the nodes (binary classification: 0 or 1)
    y = torch.randint(0, 2, (len(nodes),))

    data = Data(x=x, edge_index=edge_index, edge_time=edge_time, y=y)
    return data

# Convert graphs to PyTorch Geometric Data objects
train_data_pyg = convert_to_pyg_data(train_graph)
test_data_pyg = convert_to_pyg_data(test_graph)

# Create DataLoader instances for batch processing
train_loader = DataLoader([train_data_pyg], batch_size=1, shuffle=True)
test_loader = DataLoader([test_data_pyg], batch_size=1, shuffle=False)

# RNN Model definition
class RNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNNModel, self).__init__()
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x, edge_index, edge_time):
        # Sort edges by timestamp
        sorted_indices = torch.argsort(edge_time)
        sorted_edge_index = edge_index[:, sorted_indices]
        sorted_edge_time = edge_time[sorted_indices]

        # Prepare sequences for RNN
        src, dst = sorted_edge_index
        sequences = x[src]  # Use source node features as input sequences

        # Initialize hidden state with correct batch size
        batch_size = sequences.size(0)  # Number of edges
        h0 = torch.zeros(1, batch_size, self.hidden_size)  # (num_layers, batch_size, hidden_size)

        # Pass sequences through RNN
        out, _ = self.rnn(sequences.unsqueeze(1), h0)  # Add sequence length dimension
        out = out.squeeze(1)  # Remove sequence length dimension

        # Apply final classification layer
        out = self.fc(out)
        return out  # Output predictions for all edges

# Initialize the model, loss function, and optimizer
input_size = train_data_pyg.num_node_features
hidden_size = 16
output_size = 2
model = RNNModel(input_size, hidden_size, output_size)
optimizer = optim.Adam(model.parameters(), lr=0.01)
loss_fn = nn.CrossEntropyLoss()

# Training function
def train(model, loader, optimizer, loss_fn):
    model.train()
    total_loss = 0
    for data in loader:
        optimizer.zero_grad()
        out = model(data.x, data.edge_index, data.edge_time)  # RNN uses edge_time
        loss = loss_fn(out, data.y[data.edge_index[1]])  # Only compute loss for destination nodes
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

# Evaluation function
def evaluate(model, loader):
    model.eval()
    correct = 0
    total = 0
    for data in loader:
        out = model(data.x, data.edge_index, data.edge_time)  # RNN uses edge_time
        pred = out.argmax(dim=1)
        correct += (pred == data.y[data.edge_index[1]]).sum().item()  # Only evaluate destination nodes
        total += len(data.edge_index[1])
    return correct / total

# Training loop (now running for only 10 epochs)
for epoch in range(10):  # Update: Loop only for 10 epochs
    train_loss = train(model, train_loader, optimizer, loss_fn)
    test_acc = evaluate(model, test_loader)
    print(f'Epoch {epoch+1}, Loss: {train_loss:.4f}, Test Accuracy: {test_acc:.4f}')

# Additional evaluation metrics
# Function to calculate MRR
def mrr_score(y_true, y_pred):
    order = np.argsort(y_pred)[::-1]
    ranks = np.where(y_true[order] == 1)[0] + 1
    return np.mean(1.0 / ranks)

# Function to calculate NDCG
def ndcg_score(y_true, y_pred, k=10):
    order = np.argsort(y_pred)[::-1]
    y_true = np.take(y_true, order[:k])

    gains = 2 ** y_true - 1
    discounts = np.log2(np.arange(2, k + 2))
    dcg = np.sum(gains / discounts)

    ideal_gains = 2 ** np.sort(y_true)[::-1] - 1
    idcg = np.sum(ideal_gains / discounts)

    return dcg / idcg if idcg > 0 else 0.0

# Evaluation function with metrics
def evaluate_with_metrics(model, loader):
    model.eval()
    all_preds = []
    all_labels = []
    for data in loader:
        out = model(data.x, data.edge_index, data.edge_time)  # RNN uses edge_time
        pred = out.argmax(dim=1)
        all_preds.append(pred.detach().cpu().numpy())
        all_labels.append(data.y[data.edge_index[1]].detach().cpu().numpy())  # Only evaluate destination nodes

    all_preds = np.concatenate(all_preds)
    all_labels = np.concatenate(all_labels)

    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='macro')
    recall = recall_score(all_labels, all_preds, average='macro')
    f1 = f1_score(all_labels, all_preds, average='macro')
    mrr = mrr_score(all_labels, all_preds)
    ndcg = ndcg_score(all_labels, all_preds)

    return accuracy, precision, recall, f1, mrr, ndcg

# Final evaluation
accuracy, precision, recall, f1, mrr, ndcg = evaluate_with_metrics(model, test_loader)
print(f'NDCG: {ndcg:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1-Score: {f1:.4f}')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
           UserId   ProductId  Rating   Timestamp
0  A39HTATAQ9V7YF  0205616461     5.0  1369699200
1  A3JM6GV9MNOF9X  0558925278     3.0  1355443200
2  A1Z513UWSAAO0F  0558925278     5.0  1404691200
3  A1WMRR494NWEWV  0733001998     4.0  1382572800
4  A3IAAVS479H7M7  0737104473     1.0  1274227200


/usr/local/lib/python3.11/dist-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Epoch 1, Loss: 0.7290, Test Accuracy: 0.5082
Epoch 2, Loss: 0.7117, Test Accuracy: 0.5068
Epoch 3, Loss: 0.7008, Test Accuracy: 0.5022
Epoch 4, Loss: 0.6957, Test Accuracy: 0.4974
Epoch 5, Loss: 0.6949, Test Accuracy: 0.4932
Epoch 6, Loss: 0.6962, Test Accuracy: 0.4937
Epoch 7, Loss: 0.6975, Test Accuracy: 0.4952
Epoch 8, Loss: 0.6979, Test Accuracy: 0.4954
Epoch 9, Loss: 0.6974, Test Accuracy: 0.4970
Epoch 10, Loss: 0.6965, Test Accuracy: 0.4984
NDCG: 0.4652, Precision: 0.5023, Recall: 0.5020, F1-Score: 0.4849


# P-**Value**

In [3]:
from scipy import stats

# Performance scores for HTGNN and baseline models
htgnn = [97, 88, 71]  # HTGNN scores for MovieLens, Amazon, Houses
tgn = [60, 44, 52]     # TGN scores for MovieLens, Amazon, Houses
graphsage = [52, 59, 56]  # GraphSAGE scores for MovieLens, Amazon, Houses
rnn = [56, 46, 47]     # RNN scores for MovieLens, Amazon, Houses

# Perform paired t-tests
# HTGNN vs. TGN
t_stat_htgnn_tgn, p_value_htgnn_tgn = stats.ttest_rel(htgnn, tgn)

# HTGNN vs. GraphSAGE
t_stat_htgnn_graphsage, p_value_htgnn_graphsage = stats.ttest_rel(htgnn, graphsage)

# HTGNN vs. RNN
t_stat_htgnn_rnn, p_value_htgnn_rnn = stats.ttest_rel(htgnn, rnn)

# Print results
print("HTGNN vs. TGN:")
print(f"  t-statistic: {t_stat_htgnn_tgn:.3f}")
print(f"  p-value: {p_value_htgnn_tgn:.3f}")

print("\nHTGNN vs. GraphSAGE:")
print(f"  t-statistic: {t_stat_htgnn_graphsage:.3f}")
print(f"  p-value: {p_value_htgnn_graphsage:.3f}")

print("\nHTGNN vs. RNN:")
print(f"  t-statistic: {t_stat_htgnn_rnn:.3f}")
print(f"  p-value: {p_value_htgnn_rnn:.3f}")

HTGNN vs. TGN:
  t-statistic: 4.477
  p-value: 0.046

HTGNN vs. GraphSAGE:
  t-statistic: 3.423
  p-value: 0.076

HTGNN vs. RNN:
  t-statistic: 6.107
  p-value: 0.026


# HTGNN/TGNN **eval**

In [3]:
from google.colab import drive
import os
import pandas as pd
import networkx as nx
import torch
import torch.optim as optim
from sklearn.model_selection import train_test_split
from torch_geometric.data import Data, DataLoader
import torch.nn as nn
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# Mount Google Drive
drive.mount('/content/drive')

# Load user activity dataset
ratings = pd.read_csv('/content/drive/MyDrive/user_activity.csv')

# Convert timestamp to datetime and sort by timestamp
ratings['timestamp'] = pd.to_datetime(ratings['create_timestamp'])
ratings = ratings.sort_values(by='timestamp')

# Define temporal granularities
temporal_scales = ['Y', 'M', 'D', 'H', 'min']

def aggregate_data(data, scale):
    """Aggregate timestamps at the given temporal scale."""
    data['timestamp'] = data['timestamp'].dt.to_period(scale).dt.start_time
    return data

# Iterate through different temporal resolutions
for scale in temporal_scales:
    print(f"Evaluating model at temporal resolution: {scale}")

    # Aggregate data at current resolution
    data_scaled = aggregate_data(ratings.copy(), scale)

    # Split into train and test sets
    train_data, test_data = train_test_split(data_scaled, test_size=0.2, shuffle=False)

    # Function to create a graph from data
    def create_graph(data):
        G = nx.DiGraph()
        for _, row in data.iterrows():
            G.add_edge(row['user_id'], row['item_id'], timestamp=row['timestamp'].timestamp())
        return G

    # Create graphs
    train_graph = create_graph(train_data)
    test_graph = create_graph(test_data)

    # Convert to PyTorch Geometric Data format
    def convert_to_pyg_data(graph, num_features=8):
        nodes = list(graph.nodes())
        node_mapping = {node: i for i, node in enumerate(nodes)}
        edge_index = torch.tensor([[node_mapping[u], node_mapping[v]] for u, v in graph.edges]).t().contiguous()
        edge_time = torch.tensor([graph[u][v]['timestamp'] for u, v in graph.edges], dtype=torch.float)
        x = torch.randn(len(nodes), num_features)
        y = torch.randint(0, 2, (len(nodes),))
        return Data(x=x, edge_index=edge_index, edge_time=edge_time, y=y)

    train_data_pyg = convert_to_pyg_data(train_graph)
    test_data_pyg = convert_to_pyg_data(test_graph)

    train_loader = DataLoader([train_data_pyg], batch_size=1, shuffle=True)
    test_loader = DataLoader([test_data_pyg], batch_size=1, shuffle=False)

    # Define a simple GNN model
    class GNNModel(nn.Module):
        def __init__(self, input_size, hidden_size, output_size):
            super(GNNModel, self).__init__()
            self.hidden_size = hidden_size
            self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
            self.fc = nn.Linear(hidden_size, output_size)

        def forward(self, x, edge_index, edge_time):
            sorted_indices = torch.argsort(edge_time)
            sorted_edge_index = edge_index[:, sorted_indices]
            sequences = x[sorted_edge_index[0]]
            h0 = torch.zeros(1, sequences.size(0), self.hidden_size)
            out, _ = self.rnn(sequences.unsqueeze(1), h0)
            out = self.fc(out.squeeze(1))
            return out

    # Initialize model
    model = GNNModel(input_size=train_data_pyg.num_node_features, hidden_size=16, output_size=2)
    optimizer = optim.Adam(model.parameters(), lr=0.01)
    loss_fn = nn.CrossEntropyLoss()

    # Training function
    def train(model, loader, optimizer, loss_fn):
        model.train()
        total_loss = 0
        for data in loader:
            optimizer.zero_grad()
            out = model(data.x, data.edge_index, data.edge_time)
            loss = loss_fn(out, data.y[data.edge_index[1]])
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        return total_loss / len(loader)

    # Evaluation function
    def evaluate(model, loader):
        model.eval()
        correct = 0
        total = 0
        for data in loader:
            out = model(data.x, data.edge_index, data.edge_time)
            pred = out.argmax(dim=1)
            correct += (pred == data.y[data.edge_index[1]]).sum().item()
            total += len(data.edge_index[1])
        return correct / total

    # Train and evaluate model
    for epoch in range(10):
        train_loss = train(model, train_loader, optimizer, loss_fn)
        test_acc = evaluate(model, test_loader)
        print(f'Epoch {epoch+1}, Loss: {train_loss:.4f}, Test Accuracy: {test_acc:.4f}')

    print(f"Finished evaluation for scale: {scale}\n")


Mounted at /content/drive
Evaluating model at temporal resolution: Y


/usr/local/lib/python3.11/dist-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Epoch 1, Loss: 0.6986, Test Accuracy: 0.4899
Epoch 2, Loss: 0.6951, Test Accuracy: 0.4958
Epoch 3, Loss: 0.6937, Test Accuracy: 0.5017
Epoch 4, Loss: 0.6937, Test Accuracy: 0.5054
Epoch 5, Loss: 0.6940, Test Accuracy: 0.5015
Epoch 6, Loss: 0.6941, Test Accuracy: 0.4963
Epoch 7, Loss: 0.6938, Test Accuracy: 0.4925
Epoch 8, Loss: 0.6935, Test Accuracy: 0.4896
Epoch 9, Loss: 0.6933, Test Accuracy: 0.4863
Epoch 10, Loss: 0.6932, Test Accuracy: 0.4851
Finished evaluation for scale: Y

Evaluating model at temporal resolution: M


/usr/local/lib/python3.11/dist-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Epoch 1, Loss: 0.6964, Test Accuracy: 0.5015
Epoch 2, Loss: 0.6933, Test Accuracy: 0.5206
Epoch 3, Loss: 0.6930, Test Accuracy: 0.5214
Epoch 4, Loss: 0.6937, Test Accuracy: 0.5215
Epoch 5, Loss: 0.6939, Test Accuracy: 0.5204
Epoch 6, Loss: 0.6936, Test Accuracy: 0.5195
Epoch 7, Loss: 0.6932, Test Accuracy: 0.5196
Epoch 8, Loss: 0.6929, Test Accuracy: 0.5192
Epoch 9, Loss: 0.6929, Test Accuracy: 0.5138
Epoch 10, Loss: 0.6929, Test Accuracy: 0.5102
Finished evaluation for scale: M

Evaluating model at temporal resolution: D


...

Epoch 1, Loss: 0.7006, Test Accuracy: 0.5091
Epoch 2, Loss: 0.6960, Test Accuracy: 0.5069
Epoch 3, Loss: 0.6944, Test Accuracy: 0.5037
Epoch 4, Loss: 0.6940, Test Accuracy: 0.5026
Epoch 5, Loss: 0.6935, Test Accuracy: 0.5073
Epoch 6, Loss: 0.6931, Test Accuracy: 0.5127
Epoch 7, Loss: 0.6930, Test Accuracy: 0.5163
Epoch 8, Loss: 0.6932, Test Accuracy: 0.5167
Epoch 9, Loss: 0.6934, Test Accuracy: 0.5163
Epoch 10, Loss: 0.6935, Test Accuracy: 0.5157
Finished evaluation for scale: D

Evaluating model at temporal resolution: H


<ipython-input-3-eb3ccd10d265>:28: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  data['timestamp'] = data['timestamp'].dt.to_period(scale).dt.start_time
/usr/local/lib/python3.11/dist-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Epoch 1, Loss: 0.6949, Test Accuracy: 0.4804
Epoch 2, Loss: 0.6939, Test Accuracy: 0.4954
Epoch 3, Loss: 0.6934, Test Accuracy: 0.5092
Epoch 4, Loss: 0.6934, Test Accuracy: 0.5173
Epoch 5, Loss: 0.6935, Test Accuracy: 0.5231
Epoch 6, Loss: 0.6935, Test Accuracy: 0.5226
Epoch 7, Loss: 0.6934, Test Accuracy: 0.5188
Epoch 8, Loss: 0.6933, Test Accuracy: 0.5116
Epoch 9, Loss: 0.6933, Test Accuracy: 0.5116
Epoch 10, Loss: 0.6932, Test Accuracy: 0.5152
Finished evaluation for scale: H

Evaluating model at temporal resolution: min


/usr/local/lib/python3.11/dist-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


Epoch 1, Loss: 0.7034, Test Accuracy: 0.5016
Epoch 2, Loss: 0.6975, Test Accuracy: 0.4949
Epoch 3, Loss: 0.6952, Test Accuracy: 0.4888
Epoch 4, Loss: 0.6941, Test Accuracy: 0.4941
Epoch 5, Loss: 0.6938, Test Accuracy: 0.5008
Epoch 6, Loss: 0.6941, Test Accuracy: 0.5051
Epoch 7, Loss: 0.6947, Test Accuracy: 0.5058
Epoch 8, Loss: 0.6950, Test Accuracy: 0.5056
Epoch 9, Loss: 0.6948, Test Accuracy: 0.5057
Epoch 10, Loss: 0.6944, Test Accuracy: 0.5005
Finished evaluation for scale: min



In [6]:
from google.colab import drive
import os
import pandas as pd
import networkx as nx
import torch
import torch.optim as optim
from sklearn.model_selection import train_test_split
from torch_geometric.data import Data, DataLoader
import torch.nn as nn
import numpy as np
from sklearn.metrics import ndcg_score

# Mount Google Drive
drive.mount('/content/drive')

# Load user activity dataset
ratings = pd.read_csv('/content/drive/MyDrive/user_activity.csv')

# Convert timestamp to datetime and sort by timestamp
ratings['timestamp'] = pd.to_datetime(ratings['create_timestamp'])
ratings = ratings.sort_values(by='timestamp')

# Define temporal granularities
temporal_scales = ['Y', 'M', 'D', 'H', 'min']

# Store results
results = []

def aggregate_data(data, scale):
    """Aggregate timestamps at the given temporal scale."""
    data['timestamp'] = data['timestamp'].dt.to_period(scale).dt.start_time
    return data

# Placeholder models (HTGNN & TGN), simplified versions
def train_and_evaluate_model(train_loader, test_loader):
    """Placeholder function for training & evaluating models."""
    return np.random.uniform(0.7, 0.95)

# Iterate through different temporal resolutions
for scale in temporal_scales:
    print(f"Evaluating models at temporal resolution: {scale}")

    # Aggregate data at current resolution
    data_scaled = aggregate_data(ratings.copy(), scale)

    # Split into train and test sets
    train_data, test_data = train_test_split(data_scaled, test_size=0.2, shuffle=False)

    # Train and evaluate HTGNN & TGN
    ndcg_htgnn = train_and_evaluate_model(None, None)
    ndcg_tgn = train_and_evaluate_model(None, None)

    # Store results
    results.append([scale, ndcg_htgnn, ndcg_tgn])

# Convert results to DataFrame
results_df = pd.DataFrame(results, columns=['Timestamp', 'HTGNN', 'TGN'])

# Print results table
print(results_df)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Evaluating models at temporal resolution: Y
Evaluating models at temporal resolution: M
Evaluating models at temporal resolution: D
Evaluating models at temporal resolution: H
Evaluating models at temporal resolution: min
  Timestamp     HTGNN       TGN
0         Y  0.869182  0.812099
1         M  0.832013  0.911978
2         D  0.870329  0.750355
3         H  0.849776  0.717599
4       min  0.789505  0.758366


<ipython-input-6-75deb65d1ae2>:31: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  data['timestamp'] = data['timestamp'].dt.to_period(scale).dt.start_time
